In [4]:
import pandas as pd

def read_file(filepath):
    df = pd.read_csv(filepath, sep='\t', skiprows=[0,2,3,4,5])
    return df

df = read_file('KICH/CANCER/gdc_download_20230301_212055.202248/0ba21ef5-0829-422e-a674-d3817498c333/4868e8fc-e045-475a-a81d-ef43eabb7066.rna_seq.augmented_star_gene_counts.tsv')
df.head(10)

,gene_id,gene_name,gene_type,unstranded,stranded_first,stranded_second,tpm_unstranded,fpkm_unstranded,fpkm_uq_unstranded
0,ENSG00000000003.15,TSPAN6,protein_coding,3085,1533,1552,28.1546,12.1432,14.1998
1,ENSG00000000005.6,TNMD,protein_coding,0,0,0,0.0000,0.0000,0.0000
2,ENSG00000000419.13,DPM1,protein_coding,1896,968,928,65.0277,28.0468,32.7968
3,ENSG00000000457.14,SCYL3,protein_coding,493,373,366,2.9651,1.2789,1.4954
4,ENSG00000000460.17,C1orf112,protein_coding,107,202,201,0.7420,0.3200,0.3742
5,ENSG00000000938.13,FGR,protein_coding,624,306,318,7.6380,3.2943,3.8522
6,ENSG00000000971.16,CFH,protein_coding,935,462,473,4.8504,2.0920,2.4463
7,ENSG00000001036.14,FUCA2,protein_coding,3175,2306,2350,46.5752,20.0881,23.4902
8,ENSG00000001084.13,GCLC,protein_coding,1762,1018,1020,8.4638,3.6505,4.2687
9,ENSG00000001167.14,NFYA,protein_coding,1128,608,625,12.2529,5.2847,6.1798
